<a href="https://colab.research.google.com/github/palinkapro/ML_ZoomCamp/blob/main/Homework_unit_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import numpy as np
df = pd.read_csv('/content/drive/MyDrive/data/AB_NYC_2019.csv')

In [5]:
new_df = df[['neighbourhood_group',
             'room_type',
             'latitude', 
             'longitude', 
             'price', 
             'minimum_nights', 
             'number_of_reviews', 
             'reviews_per_month', 
             'calculated_host_listings_count', 
             'availability_365']]
             
numerical = ['latitude', 
             'longitude', 
             'minimum_nights', 
             'number_of_reviews', 
             'reviews_per_month', 
             'calculated_host_listings_count', 
             'availability_365']

categorical = ['neighbourhood_group',
               'room_type'] 

In [6]:
new_df = new_df.fillna(0)

### Question 1  

What is the most frequent observation (mode) for the column 'neighbourhood_group'?


In [7]:
print(new_df['neighbourhood_group'].mode().values)

['Manhattan']


#### Split the data

Split your data in train/val/test sets, with 60%/20%/20% distribution.  
Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.  
Make sure that the target value ('price') is not in your dataframe.  

In [8]:
from sklearn.model_selection import train_test_split

X_origin = new_df.drop(columns=['price'])
y_origin = new_df['price']
X, X_test, y, y_test = train_test_split(X_origin, y_origin, test_size=0.2, shuffle=True, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, shuffle=True, random_state=42)
X_train.shape[0] / X_origin.shape[0], X_val.shape[0] / X_origin.shape[0], X_test.shape[0] / X_origin.shape[0]

(0.6, 0.2, 0.2)

### Question 2

Create the correlation matrix for the numerical features of your train dataset.  
In a correlation matrix, you compute the correlation coefficient between every pair of features in the dataset.  
What are the two features that have the biggest correlation in this dataset?  

In [9]:
corr = X_train.corr().round(2)
corr

,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
latitude,1.00,0.08,0.03,-0.01,-0.01,0.02,-0.01
longitude,0.08,1.00,-0.06,0.06,0.13,-0.12,0.08
minimum_nights,0.03,-0.06,1.00,-0.08,-0.12,0.12,0.14
number_of_reviews,-0.01,0.06,-0.08,1.00,0.59,-0.07,0.17
reviews_per_month,-0.01,0.13,-0.12,0.59,1.00,-0.05,0.17
calculated_host_listings_count,0.02,-0.12,0.12,-0.07,-0.05,1.00,0.23
availability_365,-0.01,0.08,0.14,0.17,0.17,0.23,1.00


In [10]:
corr[corr < 1].max().sort_values(ascending=False).index[0:2]

Index(['reviews_per_month', 'number_of_reviews'], dtype='object')

#### Make price binary

We need to turn the price variable from numeric into binary.  
Let's create a variable above_average which is 1 if the price is above (or equal to) 152.  

In [11]:
new_df.price = new_df.price.apply(lambda x: 1 if x >= 152 else 0)

In [12]:
new_df

,neighbourhood_group,room_type,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,Brooklyn,Private room,40.64749,-73.97237,0,1,9,0.21,6,365
1,Manhattan,Entire home/apt,40.75362,-73.98377,1,1,45,0.38,2,355
2,Manhattan,Private room,40.80902,-73.94190,0,3,0,0.00,1,365
3,Brooklyn,Entire home/apt,40.68514,-73.95976,0,1,270,4.64,1,194
4,Manhattan,Entire home/apt,40.79851,-73.94399,0,10,9,0.10,1,0
...,...,...,...,...,...,...,...,...,...,...
48890,Brooklyn,Private room,40.67853,-73.94995,0,2,0,0.00,2,9
48891,Brooklyn,Private room,40.70184,-73.93317,0,4,0,0.00,2,36
48892,Manhattan,Entire home/apt,40.81475,-73.94867,0,10,0,0.00,1,27
48893,Manhattan,Shared room,40.75751,-73.99112,0,1,0,0.00,6,2


### Question 3

Calculate the mutual information score with the (binarized) price for the two categorical variables that we have.   Use the training set only.  
Which of these two variables has bigger score?  
Round it to 2 decimal digits using round(score, 2)  

In [13]:
from sklearn.metrics import mutual_info_score

def calculate_mi(series):
    return series.name, mutual_info_score(series, new_df.price).round(2)

mi = [calculate_mi(new_df['neighbourhood_group']), calculate_mi(new_df['room_type'])]

In [14]:
max(mi)

('room_type', 0.14)

### Question 4

Now let's train a logistic regression  
Remember that we have two categorical variables in the data. Include them using one-hot encoding.  
Fit the model on the training dataset.  
To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:  
model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)  
Calculate the accuracy on the validation dataset and rount it to 2 decimal digits.  

In [15]:
X_origin = new_df.drop(columns=['price'])
y_origin = new_df['price']
X, X_test, y, y_test = train_test_split(X_origin, y_origin, test_size=0.2, shuffle=True, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, shuffle=True, random_state=42)

In [16]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression

numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder()


preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical),
        ('cat', categorical_transformer, categorical)])
X_tr = preprocessor.fit_transform(X_train)
X_v = preprocessor.transform(X_val)

model = LogisticRegression(max_iter=10000,  solver='lbfgs', C=1.0, random_state=42)

In [17]:
model.fit(X_tr, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [18]:
acc = model.score(X_v, y_val)
acc

0.7976275692811126

### Question 5

We have 9 features: 7 numerical features and 2 categorical.  
Let's find the least useful one using the feature elimination technique.  
Train a model with all these features (using the same parameters as in Q4).  
Now exclude each feature from this set and train a model without it. Record the accuracy for each model.  
For each feature, calculate the difference between the original accuracy and the accuracy without the feature.  
Which of following feature has the smallest difference?  
neighbourhood_group  
room_type  
number_of_reviews   
reviews_per_month   
note: the difference doesn't have to be positive. 

In [19]:
feat = ['neighbourhood_group', 
        'room_type',
        'number_of_reviews',
        'reviews_per_month']


In [20]:
scores = []
for ft in feat:
  num = numerical.copy()
  cat = categorical.copy()
  if ft in num:
    num.remove(ft)
  else:
    cat.remove(ft)
  preprocessor = ColumnTransformer(
      transformers=[
          ('num', numeric_transformer, num),
          ('cat', categorical_transformer, cat)])
  X_tr = preprocessor.fit_transform(X_train.drop(columns=ft))
  X_v = preprocessor.transform(X_val.drop(columns=ft))
  model = LogisticRegression(max_iter=10000,  solver='lbfgs', C=1.0, random_state=42)
  model.fit(X_tr, y_train)
  scores.append(model.score(X_v, y_val))
  
diff = [abs(acc - x) for x in scores]
diff

[0.002658758564270336,
 0.05726556907659264,
 0.0011248593925760053,
 0.0006135596686777101]

In [21]:
feat[np.argmin(diff)]

'reviews_per_month'

### Question 6

For this question, we'll see how to use a linear regression model from Scikit-Learn. 
We'll need to use the original column 'price'. Apply the logarithmic transformation to this column.  
Fit the Ridge regression model on the training data.
This model has a parameter alpha. Let's try the following values: [0, 0.01, 0.1, 1, 10]. 
Which of these alphas leads to the best RMSE on the validation set? Round your RMSE scores to 3 decimal digits.  
If there are multiple options, select the smallest alpha.  

In [22]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

X_origin = new_df.drop(columns=['price'])
y_origin = df['price'].apply(lambda x: np.log1p(x))
X, X_test, y, y_test = train_test_split(X_origin, y_origin, test_size=0.2, shuffle=True, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, shuffle=True, random_state=42)

preprocessor = ColumnTransformer(
      transformers=[
          ('num', numeric_transformer, numerical),
          ('cat', categorical_transformer, categorical)])
X_tr = preprocessor.fit_transform(X_train)
X_v = preprocessor.transform(X_val)
alphas = [0, 0.01, 0.1, 1, 10]
rmse = []
for alpha in alphas:
  model = Ridge(alpha=alpha)
  model.fit(X_tr, y_train)
  rmse.append(round(mean_squared_error(y_val, model.predict(X_v), squared=False), 3))


In [23]:
alphas[np.argmin(rmse)]

0